In [2]:
import os
import pandas as pd
from tensorflow.keras.models import load_model
import tensorflow as tf

import sys
parent_dir = os.path.dirname(os.getcwd())
sys.path.insert(0, parent_dir)

from funciones import load_and_convert_image, get_date_time_from_image

## Carga de las Imagenes

In [17]:
ruta = 'imagenes_test' # Esta es la carpeta donde has organizado las imágenes

## Creacion del Dataset

In [18]:
# Lista para almacenar las filas del DataFrame
data = []

In [19]:
# Verificar si el archivo CSV existe
archivo_existe = os.path.exists('Muestreo_CT_PatAzul.csv')

# Si el archivo existe, cargarlo en un DataFrame
if archivo_existe:
    df_existente = pd.read_csv('Muestreo_CT_PatAzul.csv')

In [21]:
# Recorrido de los directorios
for root, dirs, files in os.walk(ruta):
    for file in files:
        if file.endswith('.JPG'):  # Asumimos que todos tus archivos tienen la extensión .JPG
            full_path = os.path.join(root, file)

            # Si el archivo CSV existe y la ruta ya está en el DataFrame, continuar con el siguiente archivo
            if archivo_existe and (df_existente['Ruta'] == full_path).any():
                continue

            parts = full_path.split(os.sep)

            # Extraer las partes basado en la estructura
            sitio = parts[-4] if len(parts) > 4 else None
            año = parts[-3] if len(parts) > 3 else None
            camara = parts[-2] if len(parts) > 2 else None
            archivo = file
            extra = None  # Por defecto, asumimos que no hay carpeta extra

            # Si hay una carpeta extra
            if len(parts) > 5:
                camara = parts[-3]
                extra = parts[-2]

            data.append([full_path, sitio, año, camara, extra, archivo])

In [22]:
# Crear un DataFrame solo si hay datos nuevos para agregar
if data:
    df = pd.DataFrame(data, columns=['Ruta', 'Sitio', 'Año', 'Camara', 'Extra', 'Archivo'])

## Extraer Fecha y Hora:

In [25]:
# Aplicar la función a cada fila del DataFrame
df['Fecha'], df['Hora'] = zip(*df['Ruta'].apply(get_date_time_from_image))
df['Fecha'] = pd.to_datetime(df['Fecha'], format="%Y:%m:%d")
df['Hora'] = pd.to_datetime(df['Hora'], format="%H:%M:%S").dt.strftime("%H:%M:%S")

In [ ]:
df.head()

,Ruta,Sitio,Año,Camara,Extra,Archivo,Fecha,Hora
0,imagenes_test/Muestreo ct Iberica/2022/Ib-22 (...,Muestreo ct Iberica,2022,Ib-22 (6),None,IMG_0212.JPG,2022-04-30,16:33:02
1,imagenes_test/Muestreo ct Iberica/2022/Ib-22 (...,Muestreo ct Iberica,2022,Ib-22 (6),None,IMG_0237.JPG,2022-04-30,17:50:59
2,imagenes_test/Muestreo ct Iberica/2022/Ib-22 (...,Muestreo ct Iberica,2022,Ib-22 (6),None,IMG_0236.JPG,2022-04-30,17:50:57
3,imagenes_test/Muestreo ct Iberica/2022/Ib-22 (...,Muestreo ct Iberica,2022,Ib-22 (6),None,IMG_0187.JPG,2022-04-30,16:18:39
4,imagenes_test/Muestreo ct Iberica/2022/Ib-22 (...,Muestreo ct Iberica,2022,Ib-22 (6),None,IMG_0186.JPG,2022-04-30,16:18:36


## Convertir a Tensor:

In [ ]:
# Aplicar la función a cada ruta de imagen en el DataFrame y guardar en una lista de tensores
image_tensors = [load_and_convert_image(img_path) for img_path in df['Ruta']]
tensor = tf.stack(image_tensors)

## Prediccion Animal vs No Animal:

In [ ]:
# Cargar el modelo desde el archivo .h5
model = load_model('../ModelosAI/ModelosFinales/modeloAnimalVGG16.h5')

In [ ]:
df['Animal_proba'] = model.predict(tensor)

4/4 [==============================] - 7s 1s/step


In [ ]:
df['Animal'] = (df['Animal_proba'] > 0.5).astype(int)

## Prediccion Guanaco vs No Guanaco:

In [ ]:
# Convertir la máscara booleana de pandas a índices numéricos
indices = [i for i, x in enumerate(df['Animal'].values) if x == 1]

# Usar tf.gather para seleccionar los elementos del tensor
tensorAnimal = tf.gather(tensor, indices)

In [30]:
# Cargar el modelo desde el archivo .h5
model = load_model('../ModelosAI/ModelosFinales/modeloGuanacoVGG16.h5')

In [ ]:
df.loc[df['Animal']==1, 'Guanaco_proba'] = model.predict(tensorAnimal)

2/2 [==============================] - 4s 2s/step


In [ ]:
df.loc[df['Animal']==1,'Guanaco'] = (df.loc[df['Animal']==1,'Guanaco_proba'] > 0.5).astype(int)

## Guardar Archivo:

In [ ]:
confianzaAnimal = 0.99
confianzaGuanaco = 0.90

In [ ]:
df['Validar'] = ((df['Animal_proba'] >= (1-confianzaAnimal)) & (df['Animal_proba'] <= confianzaAnimal)) | (df['Guanaco_proba'] <= (confianzaGuanaco))

In [ ]:
df['Validar'].value_counts()

Validar
True     50
False    50
Name: count, dtype: int64

In [ ]:
df.head()

,Ruta,Sitio,Año,Camara,Extra,Archivo,Fecha,Hora,Animal_proba,Animal,Guanaco_proba,Guanaco,Validar
0,imagenes_test/Muestreo ct Iberica/2022/Ib-22 (...,Muestreo ct Iberica,2022,Ib-22 (6),None,IMG_0212.JPG,2022-04-30,16:33:02,0.949843,1,0.787793,1.0,True
1,imagenes_test/Muestreo ct Iberica/2022/Ib-22 (...,Muestreo ct Iberica,2022,Ib-22 (6),None,IMG_0237.JPG,2022-04-30,17:50:59,0.999986,1,0.124652,0.0,True
2,imagenes_test/Muestreo ct Iberica/2022/Ib-22 (...,Muestreo ct Iberica,2022,Ib-22 (6),None,IMG_0236.JPG,2022-04-30,17:50:57,1.000000,1,0.374187,0.0,True
3,imagenes_test/Muestreo ct Iberica/2022/Ib-22 (...,Muestreo ct Iberica,2022,Ib-22 (6),None,IMG_0187.JPG,2022-04-30,16:18:39,1.000000,1,0.953375,1.0,False
4,imagenes_test/Muestreo ct Iberica/2022/Ib-22 (...,Muestreo ct Iberica,2022,Ib-22 (6),None,IMG_0186.JPG,2022-04-30,16:18:36,0.999999,1,0.679994,1.0,True


In [ ]:
df.to_csv('Muestreo_CT_PatAzul.csv')